## Import dependencies



In [0]:
import numpy as np
import pandas as pd
import pydicom
import os
import collections
import sys
import glob
import random
import cv2
import tensorflow as tf
import multiprocessing

from math import ceil, floor
from copy import deepcopy
from tqdm import tqdm
from imgaug import augmenters as iaa

import keras
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model, load_model
from keras.utils import Sequence
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

#### Install Multi-label stratification package and EfficientNet model with pretrained weights.

In [0]:
# Install Modules from internet
!pip install efficientnet
!pip install iterative-stratification

#### Input path to the data must be changed if ran locally

In [0]:
from pathlib import Path

input_path = Path("../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection")
%ls ../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/

In [0]:
# Import modules
import efficientnet.keras as efn 
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

#### Data directory must be changed if ran locally

In [0]:
# Seed
SEED = 12345
np.random.seed(SEED)
tf.set_random_seed(SEED)

# Constants
NEW_DATASET_SIZE = 0.1  # Size of the shrunk dataset
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

# Folders
DATA_DIR = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'
TEST_IMAGES_DIR = DATA_DIR + 'stage_2_test/'
TRAIN_IMAGES_DIR = DATA_DIR + 'stage_2_train/'

#### Windowing Operation (Brain, Subdural and Soft tissue windows)

In [0]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width): 
    ''' Given an dicom image, window level (WL) and window width (WW), the function calculates the upper
    and lower grey levels. Voxel values above the upper grey level will become white and below the lower
    grey level will become black. '''   

    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    
    # Resize
    img = cv2.resize(img, SHAPE[:2], interpolation = cv2.INTER_LINEAR)

    # lower grey level (y)
    img_min = window_center - window_width // 2
    # upper grey level (x)
    img_max = window_center + window_width // 2
    # Highlight the voxels above x and darken the voxels below y
    img = np.clip(img, img_min, img_max)
    return img

def bsb_window(dcm):
    ''' Create the three types of windows and stack them as the RGB channels '''
    # Brain matter window
    brain_img = window_image(dcm, 40, 80)
    # Blood/subdural window
    subdural_img = window_image(dcm, 80, 200)
    # Soft tissue window
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380

    # Stack the windows as RGB channels.
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    return bsb_img

def _read(path, SHAPE):
    dcm = pydicom.dcmread(path)
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(SHAPE)
    return img

#### Train Data generator (real-time augmentation and windows as channels)

In [0]:
# Image Augmentation
sometimes = lambda aug: iaa.Sometimes(0.25, aug)
augmentation = iaa.Sequential([ iaa.Fliplr(0.25),
                                iaa.Flipud(0.10),
                                sometimes(iaa.Crop(px=(0, 25), keep_size = True, sample_independently = False))   
                            ], random_order = True)       
        
# Generators
class TrainDataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, labels, batch_size = 16, img_size = SHAPE, img_dir = TRAIN_IMAGES_DIR, augment = False, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(indices)
        return X, Y

    def augmentor(self, image):
        augment_img = augmentation        
        image_aug = augment_img.augment_image(image)
        return image_aug

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
        np.random.shuffle(self.indices)

    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir+ID+".dcm", self.img_size)
            if self.augment:
                X[i,] = self.augmentor(image)
            else:
                X[i,] = image
            Y[i,] = self.labels.iloc[index].values        
        return X, Y

#### Load the training set; Perform data cleaning

In [0]:
def read_trainset(filename = DATA_DIR + "stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)

    duplicates_to_remove = df[df["ID"].duplicated(keep=False)].index
    
    df = df.drop(index = duplicates_to_remove)
    
    df = df.reset_index(drop = True)    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    return df

# Read Train Dataset
train_df = read_trainset()

#### Exploration on the training set

In [0]:
train_df.shape

In [0]:
print('Total number of samples:', len(train_df))
print('Number of samples where a tumour exists:', sum(train_df.sum(axis = 1)>0))
assert train_df.Label['any'].sum() == sum(train_df.sum(axis = 1)>0)

In [0]:
# Only 15% of samples have a tumour
tot_rows = len(train_df)
print('Percentage of positive samples in train set:')
for col in train_df['Label'].columns:
    print(f'{col}: {round(100*sum(train_df.Label[col])/tot_rows, 2)}')
    
# epidural has very few samples

#### Oversample the epidural cases three times, i.e., 8x 

In [0]:
# Oversample the epidural cases three times, i.e., 8x 
for i in range(3):
    epidural_df = train_df[train_df.Label['epidural'] == 1]
    train_oversample_df = pd.concat([train_df, epidural_df])
    train_df = train_oversample_df

# Summary
print('Train Shape: {}'.format(train_df.shape))
print('Test Shape: {}'.format(test_df.shape))

In [0]:
print('Percentage of positive samples in train set:')
for col in train_df['Label'].columns:
    print(f'{col}: {round(100*sum(train_df.Label[col])/tot_rows, 2)}')

#### Model Selection. User should uncomment the model they wish to run.

In [0]:
# Model checkpoint to save model weights
def ModelCheckpointFull(model_name):
    return ModelCheckpoint(model_name, 
                            monitor = 'val_acc', 
                            verbose = 1, 
                            save_best_only = True, 
                            save_weights_only = True, 
                            mode = 'max', 
                            period = 1)

# Model selection
def create_model():
    from keras.applications import resnet_v2, resnet, vgg16, inception_v3, mobilenet_v2, xception
    K.clear_session()
    
#     base_model =  efn.EfficientNetB3(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
#     base_model = efn.EfficientNetB0(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE) # Commit V2
#     base_model =  resnet.ResNet50(include_top=False, weights='imagenet', input_shape=SHAPE, pooling='avg')
#     base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=SHAPE, pooling='avg')
#     base_model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
#     base_model =  efn.EfficientNetB5(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
#     base_model =  mobilenet_v2.MobileNetV2(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
#     base_model = xception.Xception(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = SHAPE)
    
    x = base_model.output
    y_pred = Dense(6, activation = 'sigmoid')(x)

    return Model(inputs = base_model.input, outputs = y_pred)

#### Reduce dataset size to 10% of processed dataset

In [0]:
# Multi Label Stratified Split stuff...
msss = MultilabelStratifiedShuffleSplit(n_splits = 1, test_size = NEW_DATASET_SIZE, random_state = SEED)
X = train_df.index
Y = train_df.Label.values

# Get train and test index
msss_splits = next(msss.split(X, Y))
new_dataset_idx = msss_splits[1]

#### Perform train-validation split

In [0]:
# Perform train-val 85-15 stratified split on multilabel

msss = MultilabelStratifiedShuffleSplit(n_splits = 1, test_size = 0.15, random_state = SEED)
X = train_df.iloc[new_dataset_idx].index
Y = train_df.iloc[new_dataset_idx].Label.values

# Get train and valid index
msss_splits = next(msss.split(X, Y))
train_idx = msss_splits[0]
valid_idx = msss_splits[1]

In [0]:
print('Size of training set: ', len(train_idx))
print('Size of validation set:', len(valid_idx))

#### Define the model training and execute

In [0]:
# Create Data Generators for Train and Valid
data_generator_train = TrainDataGenerator(train_df.iloc[train_idx], 
                                            train_df.iloc[train_idx], 
                                            TRAIN_BATCH_SIZE, 
                                            SHAPE,
                                            augment = True)

data_generator_val = TrainDataGenerator(train_df.iloc[valid_idx], 
                                        train_df.iloc[valid_idx], 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False)

# Create Model
model = create_model()

# Full Training Model
for base_layer in model.layers[:-1]:
    base_layer.trainable = True

LR = 0.0001

model.compile(optimizer = Adam(learning_rate = LR), 
              loss = 'binary_crossentropy',
              metrics = ['acc', tf.keras.metrics.AUC()])

# Train Model
hist = model.fit_generator(generator = data_generator_train,
                    validation_data = data_generator_val,
                    epochs = 10,
                    callbacks = [ModelCheckpointFull('effb3.h5')],
                    verbose = 1)

#### Plot the accuracy and loss curves

In [0]:
history = hist
import matplotlib.pyplot as plt


fig = plt.figure(figsize = (8, 6))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy curve', fontsize = 18)
plt.ylabel('accuracy', fontsize = 16)
plt.xlabel('epoch', fontsize = 16)
plt.legend(['train', 'val'], loc='upper left')
# plt.show()
fig.savefig('acc.png', bbox_inches = 'tight')

fig = plt.figure(figsize = (8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss curve',fontsize = 18)
plt.ylabel('loss', fontsize = 16)
plt.xlabel('epoch', fontsize = 16)
plt.legend(['train', 'val'], loc='upper left')
# plt.show()
fig.savefig('loss.png', bbox_inches = 'tight')